In [ ]:
import json
import os

import pandas as pd
from dotenv import load_dotenv

from src.api_utils import get_statement, process_statement_data, send_request
from src.transformations import (
    add_cum_sum_rows,
    filter_and_group_data,
    get_filtered_data,
    get_filtered_pcr,
)
from src.visualizations import create_initial_plot, generate_buttons

pd.options.mode.copy_on_write = True

In [ ]:
load_dotenv()
token = os.getenv("TOKEN")
if token is None:
    raise ValueError("Please set the TOKEN environment variable.")
query_id = os.getenv("QUERY_ID")
if query_id is None:
    raise ValueError("Please set the QUERY_ID environment variable.")
flex_version = os.getenv("FLEX_VERSION")
if flex_version is None:
    raise ValueError("Please set the FLEX_VERSION environment variable.")
headers_str = os.getenv("HEADERS")
if headers_str is None:
    raise ValueError("Please set the HEADERS environment variable.")
headers = json.loads(headers_str)

### Get Statement from IB via API

In [ ]:
# API Calls
reference_code = send_request(token, query_id, flex_version, headers)
df = get_statement(token, reference_code, flex_version, headers)

# Process Data
df = process_statement_data(df)

### Visualize Results

In [ ]:
# filtered_df is needed for PCR later
filtered_df, grouped_df = filter_and_group_data(
    df,
    assets_to_include=None,
    symbols_to_include=["SPX"],
    notes_to_exclude=["Ex", "A"],
    dates_to_exclude=["20250102"],
)

In [ ]:
grouped_df = add_cum_sum_rows(grouped_df)
grouped_df

In [ ]:
buttons = generate_buttons(grouped_df, get_filtered_data)

In [ ]:
fig = create_initial_plot(grouped_df, buttons)
fig

### PCR

In [ ]:
result = get_filtered_pcr(filtered_df, "total")

In [ ]:
for w in sorted(grouped_df["week"].unique()):
    print(f"Woche {w}")
    result = get_filtered_pcr(filtered_df, "week", w)